In [ ]:
# Gururangan S, Marasović A, Swayamdipta S, et al. 
# Don't Stop Pretraining: Adapt Language Models to Domains and Tasks[J]. arXiv preprint arXiv:2004.10964, 2020.

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/root/data/transformers/model_zoo'

from tqdm import tqdm
from pathlib import Path
from transformers import (
    AlbertForMaskedLM,
    BertTokenizer,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    Trainer,
    TrainingArguments
)

In [ ]:
data_path = Path('../../data/tnews/')

In [ ]:
pretrained = 'voidful/albert_chinese_tiny'
tokenizer = BertTokenizer.from_pretrained(pretrained, mirror='tuna')
model = AlbertForMaskedLM.from_pretrained(pretrained, mirror='tuna')
pretrain_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=data_path / 'pretrain_data.txt',
    block_size=512
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=pretrain_dataset
)

trainer.train()

In [ ]:
trainer.save_model('./dapt_model')
tokenizer.save_pretrained('./dapt_model/')